In [1]:
import numpy as np
import cv2
import os
import cvlib as cv
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from PIL import Image, ImageOps
import cv2
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split

In [2]:
#Loading All Saved Models

gender_model = models.load_model('gender.h5')
age_model = models.load_model('agemodel.h5')
race_model = models.load_model('race.h5')

In [ ]:
#Flask and Webcam IMPLEMENTATION FOR REAL TIME PREDICTIONS ON LIVE SERVER
app=Flask(__name__)
camera=cv2.VideoCapture(0)

def face_detection():
    while True:
            
        ## read the camera frame
        success,frame=camera.read()
        if not success:
            break
        else:
            # apply face detection
            face, confidence = cv.detect_face(frame)


            # loop through detected faces
            for idx, f in enumerate(face):

                # get corner points of face rectangle        
                (startX, startY) = f[0], f[1]
                (endX, endY) = f[2], f[3]

                # draw rectangle over face
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
                
                
                # crop the detected face region
                face_crop = np.copy(frame[startY:endY,startX:endX])
                face_crop = cv2.resize(face_crop, (200, 200))
                face_crop = face_crop / 255.0
                face_crop = np.expand_dims(face_crop, axis = 0)
                
                #predictions
                gen_pred = gender_model.predict(face_crop)
                age_pred = age_model.predict(face_crop)
                age_pred = np.argmax(age_pred)
                race_pred = race_model.predict(face_crop)
                race_pred = np.argmax(race_pred)

                if gen_pred >= 0.5:
                    gen_pred = 'female'
                else:
                    gen_pred = 'male'
                    
        # write labels and confidence above face rectangle
                label = gen_pred,age_pred,race_pred
                cv2.putText(frame, label, (startX, startY),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

            ret,buffer=cv2.imencode('.jpg',frame)
            frame=buffer.tobytes()

        yield(b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video')
def video():
    return Response(face_detection(),mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__=="__main__":
    app.run(debug=True)